# Decomposing image information
Stough, DIP

It can be valuable in compression to rethink of an image in terms of important and less important information. At the very core in this example, we recode each pair of pixels as a mean, difference. Then if the difference is quite small, it can potentially be ignored.

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

#Probably shouldn't have to repeat this in every script.
#https://matplotlib.org/users/dflt_style_changes.html
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'gray'

## A very simple basis for any pair of pixels.

In [2]:
phi = np.array([1, 1])/np.sqrt(2)
psi = np.array([1, -1])/np.sqrt(2)

In [3]:
#Decompose each row into a half-row of phi's and a half-row of psi's
#Assuming a multiple of 2 on the number of columns.
def haarDecompOnce(I, keep):
    #keep between 0 and 100
    
    Ic = np.zeros(I.shape)
    for c in range(3):

        X = np.reshape(I[...,c].ravel(order='C'), (2, I[...,c].size//2), order='F')
        phis = np.inner(phi, X.transpose())
        phisC = np.zeros(phis.shape)

        
        cutoff = np.percentile(np.abs(phis.ravel()), keep)
        #print(cutoff)
        mask = np.abs(phis) >= cutoff
        phisC[mask] = phis[mask]
        #print(phisC.shape)
        
        psis = np.inner(psi, X.transpose())
        psisC = np.zeros(psis.shape)
        
        cutoff = np.percentile(np.abs(psis.ravel()), keep)
        mask = np.abs(psis) >= cutoff
        psisC[mask] = psis[mask]
        
        
        Ic[...,c] = np.concatenate([np.reshape(phisC, (I.shape[0], I.shape[1]//2)),
                           np.reshape(psisC, (I.shape[0], I.shape[1] // 2))],
                          axis=1)

    
    return Ic

def haarDecomp(I):
    if not np.all([np.log2(d).is_integer() for d in I.shape[:2]]):
        raise ValueError('haarDecomp: requires image with dimensions powers of 2.')

    howmany = min([int(np.log2(d)) for d in I.shape[:2]])

    J = I.copy()
    for h in range(howmany):
        J = haarDecompOnce(J)
        J = haarDecompOnce(J.transpose())
    return J

#Left for later...
def haarRecompOnce(J):
    Jcd = np.zeros(J.shape)
    for c in range(3):
        Xleft = J[:,:J.shape[1]//2,c] #mean
        Xright = J[:,J.shape[1]//2:,c] #difference
        
        Xnew = np.stack((Xleft.ravel(),Xright.ravel()))
        phis = np.inner(phi, Xnew.transpose()) #x
        psis = np.inner(psi, Xnew.transpose()) #y
            
        j = 0
        for i in range(len(phis)):
            Jcd[j//512,j%512,c] = phis[i]
            j = j + 1
            Jcd[j//512,j%512,c] = psis[i]
            j = j + 1
    return Jcd

def haarRecomp(I):
    if not np.all([np.log2(d).is_integer() for d in I.shape[:2]]):
        raise ValueError('haarDecomp: requires image with dimensions powers of 2.')

    howmany = min([int(np.log2(d)) for d in I.shape[:2]])

    J = I.copy()
    for h in range(howmany):
        J = haarRecompOnce(J)
        J = haarRecompOnce(J.transpose())
    return J
    

In [4]:
#Just going to do single channel here.
I = plt.imread('cat_small.png').astype('float')[...,:3]
#GI = 0.2989 * I[..., 0] + 0.5870 * I[..., 1] + 0.1140 * I[..., 2]
I = I/I.max()

#k = 2^p + q -1
#Code to pad to the nearest power of two
dimsRounded = [int(np.power(2, np.ceil(np.log2(d)))) for d in I.shape[:2]] ## p

#Padded to the power of 2.
CI = np.zeros(I.shape)
for c in range(3):
    CI[...,c] = np.pad(I[...,c],
                       pad_width=((0, dimsRounded[0]-I[...,c].shape[0]),
                                  (0, dimsRounded[1]-I[...,c].shape[1])),
                       mode='constant', constant_values=0)




First = haarDecompOnce(CI,0)
#for c in range(3):
##    First[...,c] = First[...,c]-First[...,c].min()
#    First[...,c] = First[...,c]/First[...,c].max()

FirstT = np.zeros(First.shape)
for c in range(3):
    FirstT[...,c] = First[...,c].transpose()

Second = haarDecompOnce(FirstT,0)

for c in range(3):
    Second[...,c] = Second[...,c].transpose()

f, ax = plt.subplots(1,3, figsize=(10,3))

ax[0].imshow(CI)
ax[0].set_title('Original')

ax[1].imshow(np.clip(First,0,1))
ax[1].set_title('Step 1: Row Transform')

ax[2].imshow(np.clip(Second,0,1))
ax[2].set_title('Step 2: Column Transform')

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
def Trans(I):
    T = I.copy()
    for c in range(3):
        T[...,c] = T[...,c].transpose()
    return T

f, ax = plt.subplots(1,4, figsize=(10,3))

ax[0].imshow(CI)
ax[0].set_title('Original')

ax[1].imshow(haarRecompOnce(First))
ax[1].set_title('One recomp')

D = haarRecompOnce(Second)

ax[2].imshow(haarRecompOnce(Second))
ax[2].set_title('frist of two recomps')

SecondT = np.zeros(Second.shape)

    
    
    
ax[3].imshow(haarRecompOnce(Trans(haarRecompOnce(Trans(Second)))))
ax[3].set_title('Second recomp')

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [6]:
#Just going to do single channel here.
I = plt.imread('cat_small.png').astype('float')[...,:3]
#GI = 0.2989 * I[..., 0] + 0.5870 * I[..., 1] + 0.1140 * I[..., 2]
I = I/I.max()

#k = 2^p + q -1
#Code to pad to the nearest power of two
dimsRounded = [int(np.power(2, np.ceil(np.log2(d)))) for d in I.shape[:2]] ## p

#Padded to the power of 2.
CI = np.zeros(I.shape)
for c in range(3):
    CI[...,c] = np.pad(I[...,c],
                       pad_width=((0, dimsRounded[0]-I[...,c].shape[0]),
                                  (0, dimsRounded[1]-I[...,c].shape[1])),
                       mode='constant', constant_values=0)




First = haarDecompOnce(CI,10) #keeping 90%
#for c in range(3):
##    First[...,c] = First[...,c]-First[...,c].min()
#    First[...,c] = First[...,c]/First[...,c].max()
f, ax = plt.subplots(1,3, figsize=(10,3))
ax[0].imshow(CI)
ax[0].set_title('Original')

ax[1].imshow(np.clip(First,0,1))
ax[1].set_title('Harr decomp keeping 90%')

ax[2].imshow(np.clip(haarRecompOnce(First),0,1))
ax[2].set_title('Harr Recomp')

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Part B

In [7]:
from skimage.util import *

In [8]:
l = plt.imread("eye.png")[...,:3]

In [9]:
l.shape
I = l[:-1,:-1,:]

In [10]:
block_shape = (8,8,3)

view = view_as_blocks(I, block_shape=block_shape)
view = np.squeeze(view)
blockView = view.reshape([view.shape[0]*view.shape[1]] + list(view.shape[2:]))


In [11]:
newImgBlock1 = np.zeros(blockView.shape)
newImgBlock2 = np.zeros(blockView.shape)
newImgBlock3 = np.zeros(blockView.shape)

for i, block in enumerate(blockView):
    bT = np.mean(block, axis=(0,1)) 
    newImgBlock1[i][:] = np.reshape(bT, (1,1,3))

for i, block in enumerate(blockView):
    bT = np.min(block, axis=(0,1)) 
    newImgBlock2[i][:] = np.reshape(bT, (1,1,3))
    
for i, block in enumerate(blockView):
    bT = np.median(block, axis=(0,1)) 
    newImgBlock3[i][:] = np.reshape(bT, (1,1,3))
    

In [12]:
Imean = montage(newImgBlock1, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)
Imin = montage(newImgBlock2, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)
Imedian = montage(newImgBlock3, grid_shape=[view.shape[0], view.shape[1]], multichannel=True)

In [13]:
f, ax = plt.subplots(1,4, figsize=(10,3))


ax[0].imshow(I)
ax[0].set_title('Original')

ax[1].imshow(Imean)
ax[1].set_title('Mean Blocks')

ax[2].imshow(Imin)
ax[2].set_title('Min Blocks')

ax[3].imshow(Imedian)
ax[3].set_title('Median Blocks')

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …